In [1]:
from IPython.display import display, Markdown

import pandas as pd
import gc
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score

from sklearn.metrics import roc_auc_score

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

from sklearn.tree import plot_tree

from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler, OneHotEncoder

from scipy.sparse import hstack

# XGBoost
import xgboost as xgb

import numpy as np

import matplotlib.pyplot as plt

import itertools


In [2]:
menores_a_50 = []
comp_data = pd.read_csv("./competition_data.csv")
for i in range(len(comp_data.columns)):
    if (len(comp_data[comp_data.columns[i]].unique()) < 50):
        menores_a_50.append(comp_data.columns[i])
        print(i,comp_data.columns[i], len(comp_data[comp_data.columns[i]].unique()))
print(len(comp_data))
print(menores_a_50)

0 accepts_mercadopago 1
8 benefit 3
9 boosted 1
11 conversion 3
12 date 31
15 etl_version 1
16 free_shipping 2
17 fulfillment 2
19 health 35
20 is_pdp 3
23 listing_type_id 2
24 logistic_type 7
28 platform 4
34 site_id 1
199972
['accepts_mercadopago', 'benefit', 'boosted', 'conversion', 'date', 'etl_version', 'free_shipping', 'fulfillment', 'health', 'is_pdp', 'listing_type_id', 'logistic_type', 'platform', 'site_id']


In [13]:
comp_data['ROW_ID'].count()

19211

In [3]:
len(comp_data)

199972

In [4]:
comp_data.head()

,accepts_mercadopago,available_quantity,avg_gmv_item_domain_30days,avg_gmv_item_sel,avg_gmv_seller_bday,avg_qty_orders_item_domain_30days,avg_qty_orders_item_sel_30days,avg_si_item_sel_30day,benefit,boosted,...,total_si_domain_30days,total_si_item_30days,total_si_sel_30days,total_visits_domain,total_visits_item,total_visits_seller,uid,user_id,warranty,ROW_ID
0,True,59,938.031376,2153.933091,135195.200333,3.430563,12.407860,13.691450,0.001,False,...,56717.0,17.0,25781.0,7993972,543,854813,cb5bfac7-229a-4e3e-96dd-80b2ad3972c1,NaN,Garantía de fábrica: 12 meses,NaN
1,True,37,159.826531,240.325973,19338.230000,3.142739,4.429577,4.511599,0.001,False,...,81455.0,5.0,10891.0,7821316,1789,1422288,b0f9aa19-00d5-4afd-aa77-45be7c849562,NaN,NaN,NaN
2,True,9,67.635391,86.066296,154.919333,2.633771,2.592593,2.722222,0.001,False,...,16792.0,8.0,147.0,1303909,1473,17801,fd59890a-f2c2-4282-b2ee-c68f25e20697,151157147.0,NaN,NaN
3,True,12,92.085756,332.848400,554.747333,3.280069,12.620000,12.840000,0.001,False,...,1988.0,14.0,642.0,139419,7473,44336,72340acf-2a1c-4214-a604-132e0cb56939,NaN,Garantía de fábrica: 90 días,NaN
4,True,5,156.905222,955.997475,3154.791667,2.737876,3.666667,4.363636,0.001,False,...,8707.0,32.0,432.0,209373,416,46206,84da1314-814c-4d4f-be23-3de9ea1da4b6,NaN,Garantía del vendedor: 90 días,NaN


In [5]:
comp_data.describe()

,available_quantity,avg_gmv_item_domain_30days,avg_gmv_item_sel,avg_gmv_seller_bday,avg_qty_orders_item_domain_30days,avg_qty_orders_item_sel_30days,avg_si_item_sel_30day,benefit,conversion,health,...,total_orders_item_30days,total_orders_sel_30days,total_si_domain_30days,total_si_item_30days,total_si_sel_30days,total_visits_domain,total_visits_item,total_visits_seller,user_id,ROW_ID
count,199972.000000,195607.000000,195607.000000,195607.000000,195607.000000,195607.000000,195607.000000,199971.000000,180761.000000,199972.000000,...,195607.000000,195607.000000,195607.000000,195607.000000,195607.000000,1.999720e+05,199972.000000,1.999720e+05,1.783750e+05,19211.000000
mean,2142.683766,333.141264,1160.099822,10815.338962,4.385187,13.542561,18.286236,0.000976,0.092631,0.809401,...,65.176297,3941.158614,29197.693135,88.487667,5238.731257,2.170257e+06,5571.603790,2.542803e+05,2.246094e+08,9605.000000
std,11326.386712,492.836951,4284.246365,26377.753704,2.768258,25.645743,33.648431,0.000154,0.289915,0.146368,...,129.610777,5145.727129,49694.994203,220.606860,6809.183739,3.545316e+06,12108.897095,3.458552e+05,1.474232e+08,5545.882346
min,0.000000,4.909057,4.773333,0.660000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,4.000000,1.000000,2.000000,3.160000e+02,1.000000,1.100000e+01,1.610000e+02,0.000000
25%,19.000000,86.729688,143.184865,793.281667,2.876796,4.188679,4.916896,0.001000,0.000000,0.770000,...,6.000000,718.000000,3938.000000,7.000000,943.000000,1.899680e+05,407.000000,4.804600e+04,1.053200e+08,4802.500000
50%,67.000000,169.234106,262.266733,2733.067000,3.639912,6.768293,8.875000,0.001000,0.000000,0.810000,...,20.000000,2102.000000,11660.000000,24.000000,2713.000000,5.613990e+05,1496.000000,1.263900e+05,1.976426e+08,9605.000000
75%,371.000000,319.281502,675.989333,9217.855667,5.066384,13.403873,18.558559,0.001000,0.000000,0.900000,...,63.000000,5017.000000,38725.000000,80.000000,6455.000000,2.007145e+06,5248.000000,2.817150e+05,3.202613e+08,14407.500000
max,99999.000000,3778.257881,57282.961233,244988.442667,38.148629,258.363636,465.500000,0.001000,1.000000,1.000000,...,1853.000000,36492.000000,505258.000000,8269.000000,56189.000000,1.591114e+07,195413.000000,2.938795e+06,1.000014e+09,19210.000000


In [6]:
comp_data.describe(include = 'object')

,category_id,date,deal_print_id,domain_id,etl_version,full_name,is_pdp,item_id,listing_type_id,logistic_type,main_picture,platform,print_server_timestamp,site_id,tags,title,uid,warranty
count,199972,199972,199972,199972,199972,199972,109788,199972,199972,199972,196141,199972,199972,199972,199972,199972,199972,152281
unique,2284,31,196744,1742,1,2280,2,27695,2,7,26961,4,196780,1,1227,26745,159996,1288
top,MLA109027,2020-04-25,308af510-82c6-11ea-bf54-5d2729201b69,MLA-SNEAKERS,0.0.1,Ropa y Accesorios -> Calzado -> Zapatillas,False,MLA784334044,gold_special,cross_docking,http://mla-s1-p.mlstatic.com/896280-MLA3240749...,/mobile/android,2020-04-20T23:09:08.671-0400,MLA,"[brand_verified, good_quality_picture, good_qu...",Auriculares Bluetooth Inalambricos In Ear Con ...,0,Sin garantía
freq,12333,16497,21,18203,199972,12333,109136,717,165219,108346,717,124781,9,199972,29687,731,50,23632


In [7]:
comp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199972 entries, 0 to 199971
Data columns (total 57 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   accepts_mercadopago                199972 non-null  bool   
 1   available_quantity                 199972 non-null  int64  
 2   avg_gmv_item_domain_30days         195607 non-null  float64
 3   avg_gmv_item_sel                   195607 non-null  float64
 4   avg_gmv_seller_bday                195607 non-null  float64
 5   avg_qty_orders_item_domain_30days  195607 non-null  float64
 6   avg_qty_orders_item_sel_30days     195607 non-null  float64
 7   avg_si_item_sel_30day              195607 non-null  float64
 8   benefit                            199971 non-null  float64
 9   boosted                            199972 non-null  bool   
 10  category_id                        199972 non-null  object 
 11  conversion                         1807

In [14]:
# Assuming 'comp_data' is your DataFrame
data = []

for i, column_name in enumerate(comp_data.columns):
    # if isinstance(comp_data[column_name][0], (float, int)) and column_name in menores_a_50:
        data.append([ column_name, type(comp_data[column_name][0])])
#print(atributos_ohe)

# Create a DataFrame from the data
table_df = pd.DataFrame(data, columns=['Column Name', 'Data Type'])

# Display the DataFrame as a table
print(table_df)

                          Column Name                Data Type
0                 accepts_mercadopago    <class 'numpy.bool_'>
1                  available_quantity    <class 'numpy.int64'>
2          avg_gmv_item_domain_30days  <class 'numpy.float64'>
3                    avg_gmv_item_sel  <class 'numpy.float64'>
4                 avg_gmv_seller_bday  <class 'numpy.float64'>
5   avg_qty_orders_item_domain_30days  <class 'numpy.float64'>
6      avg_qty_orders_item_sel_30days  <class 'numpy.float64'>
7               avg_si_item_sel_30day  <class 'numpy.float64'>
8                             benefit  <class 'numpy.float64'>
9                             boosted    <class 'numpy.bool_'>
10                        category_id            <class 'str'>
11                         conversion  <class 'numpy.float64'>
12                               date            <class 'str'>
13                      deal_print_id            <class 'str'>
14                          domain_id            <class

In [9]:
comp_data.head()

,accepts_mercadopago,available_quantity,avg_gmv_item_domain_30days,avg_gmv_item_sel,avg_gmv_seller_bday,avg_qty_orders_item_domain_30days,avg_qty_orders_item_sel_30days,avg_si_item_sel_30day,benefit,boosted,...,total_si_domain_30days,total_si_item_30days,total_si_sel_30days,total_visits_domain,total_visits_item,total_visits_seller,uid,user_id,warranty,ROW_ID
0,True,59,938.031376,2153.933091,135195.200333,3.430563,12.407860,13.691450,0.001,False,...,56717.0,17.0,25781.0,7993972,543,854813,cb5bfac7-229a-4e3e-96dd-80b2ad3972c1,NaN,Garantía de fábrica: 12 meses,NaN
1,True,37,159.826531,240.325973,19338.230000,3.142739,4.429577,4.511599,0.001,False,...,81455.0,5.0,10891.0,7821316,1789,1422288,b0f9aa19-00d5-4afd-aa77-45be7c849562,NaN,NaN,NaN
2,True,9,67.635391,86.066296,154.919333,2.633771,2.592593,2.722222,0.001,False,...,16792.0,8.0,147.0,1303909,1473,17801,fd59890a-f2c2-4282-b2ee-c68f25e20697,151157147.0,NaN,NaN
3,True,12,92.085756,332.848400,554.747333,3.280069,12.620000,12.840000,0.001,False,...,1988.0,14.0,642.0,139419,7473,44336,72340acf-2a1c-4214-a604-132e0cb56939,NaN,Garantía de fábrica: 90 días,NaN
4,True,5,156.905222,955.997475,3154.791667,2.737876,3.666667,4.363636,0.001,False,...,8707.0,32.0,432.0,209373,416,46206,84da1314-814c-4d4f-be23-3de9ea1da4b6,NaN,Garantía del vendedor: 90 días,NaN


## ----------------------------------------------------------------
# Intento 1

LOAD DATA y drop atribute

In [16]:
def load_comp():
    # Load the competition data
    comp_data = pd.read_csv("./competition_data.csv")
    # eliminamos las columnas que no consideramos utiles para la estimacion
    comp_data = comp_data.drop(
        columns=['accepts_mercadopago',
                # 'available_quantity',
                # 'avg_gmv_item_domain_30days',
                # 'avg_gmv_item_sel',
                # 'avg_gmv_seller_bday',
                # 'avg_qty_orders_item_domain_30days',
                # 'avg_qty_orders_item_sel_30days',
                # 'avg_si_item_sel_30day',
                'benefit',
                'boosted',
                'category_id',
                # 'conversion',
                'date',
                'deal_print_id',
                # 'domain_id',
                'etl_version',
                # 'free_shipping',
                # 'fulfillment',
                'full_name',
                # 'health',
                # 'is_pdp',
                # 'product_id',
                # 'item_id',
                # 'listing_type_id',
                # 'logistic_type',
                # 'main_picture',
                # 'offset',
                # 'original_price',
                # 'platform',
                # 'price',
                # 'print_position',
                'print_server_timestamp',
                # 'qty_items_dom',
                # 'qty_items_sel',
                # 'site_id',
                # 'sold_quantity',
                'tags',
                'title',
                # 'total_asp_item_domain_30days',
                # 'total_asp_item_sel_30days',
                # 'total_gmv_domain_bday',
                # 'total_gmv_item_30days',
                # 'total_items_domain',
                # 'total_items_seller',
                # 'total_orders_domain_30days',
                # 'total_orders_item_30days',
                # 'total_orders_sel_30days',
                # 'total_si_domain_30days',
                # 'total_si_item_30days',
                # 'total_si_sel_30days',
                # 'total_visits_domain',
                # 'total_visits_item',
                # 'total_visits_seller',
                'uid',
                'user_id',
                'warranty',
                # 'ROW_ID'
                ]
    )
    # OHE comp_data columns
    comp_data = pd.get_dummies(comp_data, 
        sparse=True,
        columns=[
            # "category_id", 
            #"domain_id", 
            "logistic_type", 
            "platform", 
            "site_id"
        ],
        dtype=int
    )
    #comp_data["accepts_mercadopago"] = comp_data["accepts_mercadopago"].astype(int)
    # todos aceptan mercadopago
    # comp_data["boosted"] = comp_data["boosted"].astype(int)
    comp_data["free_shipping"] = comp_data["free_shipping"].astype(int)
    comp_data["fulfillment"] = comp_data["fulfillment"].astype(int)

    # comp_data["is_pdp"].fillna(0, inplace=True)
    # comp_data["is_pdp"] = comp_data["is_pdp"].astype(int)
    #comp_data["warranty"] = comp_data["warranty"].astype(int)

    # comp_data["listing_type_id"] to 0 if gold_special, 1 if gold_pro.
    comp_data["listing_type_id"] = comp_data["listing_type_id"].apply(lambda x: 0 if x == "gold_special" else 1)
    
    # Label encode category_id and domain_id
    # comp_data["category_id"] = comp_data["category_id"].astype("category")#.cat.codes
    #comp_data["domain_id"] = comp_data["domain_id"].astype("category")#.cat.codes

    # sklearn LabelEncoder for category_id and domain_id
    
    # comp_data["category_id"] = LabelEncoder().fit_transform(comp_data["category_id"]).astype(int)
    comp_data["domain_id"] = LabelEncoder().fit_transform(comp_data["domain_id"]).astype(int)
    comp_data["item_id"] = LabelEncoder().fit_transform(comp_data["item_id"]).astype(int)

    comp_data["price_diff"] = comp_data["price"] - comp_data["original_price"]
    comp_data["cheaper_than_original"] = comp_data["price_diff"].apply(lambda x: 1 if x < 0 else 0)
    comp_data["price_diff"] = comp_data["price_diff"].apply(lambda x: abs(x)).astype(int)

    #comp_data["cheaper_than_avg"] = comp_data["price"] - comp_data["avg_asp_item_domain"]
    #comp_data["cheaper_than_avg"] = comp_data["cheaper_than_avg"].apply(lambda x: 1 if x < 0 else 0).astype(int)

    # Drop useless columns
    # comp_data = comp_data.drop(
    #     columns=[
    #     ]
    # )

    return comp_data
comp_data = load_comp()
comp_data["item_id"].value_counts()

14540    717
11932    701
27372    622
14900    535
12183    517
        ... 
23949      1
3863       1
14492      1
18426      1
11527      1
Name: item_id, Length: 27695, dtype: int64

SPLIT DATA

In [17]:
# Split into training and evaluation samples
comp_data = load_comp()

display(
    Markdown(
        "\n".join(
            [
                "- **{}** ({})".format(col, dtype) for col, dtype in
                zip(comp_data.columns, comp_data.dtypes)
            ]
        )
    )
)

full_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]

# Get number column names for full_data
# print(full_data.columns)
# num_cols = full_data.select_dtypes(include='number').columns
# print(num_cols)
# difference between full_data_cols and num_cols
# print(set(full_data.columns) - set(num_cols))

del comp_data
gc.collect()

train_data, test_data = train_test_split(full_data, test_size=0.20, train_size=0.80, random_state=42)

y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number')

y_test = test_data["conversion"]
X_test = test_data.drop(columns=["conversion", "ROW_ID"])
X_test = X_test.select_dtypes(include='number')

del train_data
del test_data

gc.collect()

- **available_quantity** (int64)
- **avg_gmv_item_domain_30days** (float64)
- **avg_gmv_item_sel** (float64)
- **avg_gmv_seller_bday** (float64)
- **avg_qty_orders_item_domain_30days** (float64)
- **avg_qty_orders_item_sel_30days** (float64)
- **avg_si_item_sel_30day** (float64)
- **conversion** (float64)
- **domain_id** (int64)
- **free_shipping** (int64)
- **fulfillment** (int64)
- **health** (float64)
- **is_pdp** (object)
- **product_id** (float64)
- **item_id** (int64)
- **listing_type_id** (int64)
- **main_picture** (object)
- **offset** (int64)
- **original_price** (int64)
- **price** (int64)
- **print_position** (int64)
- **qty_items_dom** (float64)
- **qty_items_sel** (float64)
- **sold_quantity** (int64)
- **total_asp_item_domain_30days** (float64)
- **total_asp_item_sel_30days** (float64)
- **total_gmv_domain_bday** (float64)
- **total_gmv_item_30days** (float64)
- **total_items_domain** (int64)
- **total_items_seller** (int64)
- **total_orders_domain_30days** (float64)
- **total_orders_item_30days** (float64)
- **total_orders_sel_30days** (float64)
- **total_si_domain_30days** (float64)
- **total_si_item_30days** (float64)
- **total_si_sel_30days** (float64)
- **total_visits_domain** (int64)
- **total_visits_item** (int64)
- **total_visits_seller** (int64)
- **ROW_ID** (float64)
- **logistic_type_cross_docking** (Sparse[int64, 0])
- **logistic_type_custom** (Sparse[int64, 0])
- **logistic_type_default** (Sparse[int64, 0])
- **logistic_type_drop_off** (Sparse[int64, 0])
- **logistic_type_fulfillment** (Sparse[int64, 0])
- **logistic_type_not_specified** (Sparse[int64, 0])
- **logistic_type_xd_drop_off** (Sparse[int64, 0])
- **platform_/mobile/android** (Sparse[int64, 0])
- **platform_/mobile/ios** (Sparse[int64, 0])
- **platform_/web/desktop** (Sparse[int64, 0])
- **platform_/web/mobile** (Sparse[int64, 0])
- **site_id_MLA** (Sparse[int64, 0])
- **price_diff** (int64)
- **cheaper_than_original** (int64)

0

In [18]:
atributes_to_poly = [
                        "available_quantity", 
                        "avg_gmv_item_domain_30days", 
                        "avg_gmv_item_sel", 
                        "avg_gmv_seller_bday", 
                        "avg_qty_orders_item_domain_30days", 
                        "avg_qty_orders_item_sel_30days", 
                        "avg_si_item_sel_30day",
                        "original_price",
                        "price",
                    ]

imputed_poly_features = make_pipeline(
  #SimpleImputer(),
  #PolynomialFeatures(),
  StandardScaler(),
  xgb.XGBClassifier(
        missing=np.nan,
        n_jobs=-1,
        tree_method='gpu_hist',
    )
)

# Custom Layer Starts Here:
pl = PolynomialFeatures(2, interaction_only=True, include_bias=False)

#Input X contains NaN.

imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()
print(imputed_X_train_plus.shape[0] == X_train.shape[0])

# Make new columns indicating what will be imputed
cols_with_missing = (col for col in X_train.columns
                        if X_train[col].isnull().any())

for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()

poly_X_train = imputed_X_train_plus[atributes_to_poly]
poly_X_test = imputed_X_test_plus[atributes_to_poly]
print(poly_X_train.shape[0] == X_train.shape[0])

poly_X_train = my_imputer.fit_transform(poly_X_train)
poly_X_test = my_imputer.transform(poly_X_test)
print(poly_X_train.shape[0] == X_train.shape[0])

poly_X_train = pl.fit_transform(poly_X_train)
poly_X_test = pl.fit_transform(poly_X_test)
print(poly_X_train.shape[0] == X_train.shape[0])

new_X_train = np.concatenate((X_train.drop(atributes_to_poly, axis=1), poly_X_train), axis=1)
new_X_test = np.concatenate((X_test.drop(atributes_to_poly, axis=1), poly_X_test), axis=1)
print(new_X_train.shape[0] == X_train.shape[0])
# Custom Layer Ends Here


imputed_poly_features.fit(new_X_train, y_train)

roc_auc_score(y_test, imputed_poly_features.predict_proba(new_X_test)[:, imputed_poly_features.classes_ == 1])
# Decision Tree Classifier with K-Fold Cross Validation and Randomized Search CV

dtc = make_pipeline(
    SimpleImputer(),
    RandomizedSearchCV(
        DecisionTreeClassifier(random_state=2345),
        param_distributions={
            "max_depth": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
            "min_samples_split": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            "min_samples_leaf": [2, 4, 8, 16, 32, 64, 128, 256, 512],
            "max_features": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
        },
        n_iter=100,
        cv=3,
        random_state=2345,
        n_jobs=-1,
        verbose=0,
    )
)

kf = KFold(n_splits=5, random_state=2345, shuffle=True)
roc_aucs = []

for train_index, valid_index in kf.split(X_train):
    X_train_kf, X_valid_kf = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_kf, y_valid_kf = y_train.iloc[train_index], y_train.iloc[valid_index]

    dtc.fit(X_train_kf, y_train_kf)
    # Metric: roc_auc_score
    roc_aucs.append(roc_auc_score(y_valid_kf, dtc.predict_proba(X_valid_kf)[:, dtc.classes_ == 1]))
    print(f"ROC-AUC split {len(roc_aucs)}: ", roc_aucs[-1])

print("ROC-AUC mean: ", sum(roc_aucs) / len(roc_aucs))    

y_pred = dtc.fit(X_train, y_train)

print("ROC-AUC: ", roc_auc_score(y_test, dtc.predict_proba(X_test)[:, dtc.classes_ == 1]))

True
True
True
True
True


XGBoostError: [00:55:56] /Users/runner/work/xgboost/xgboost/src/gbm/../common/common.h:163: XGBoost version not compiled with GPU support.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000136ff6ab5 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x00000001371468c5 xgboost::gbm::GBTree::Configure(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>>, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>>>>> const&) + 1573
  [bt] (2) 3   libxgboost.dylib                    0x000000013716919d xgboost::LearnerConfiguration::Configure() + 1197
  [bt] (3) 4   libxgboost.dylib                    0x00000001371694da xgboost::

In [ ]:
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("exp_oli_1.csv", sep=",", index=False)

## ----------------------------------------------------------------
# Intento 2

In [ ]:
comp_data = pd.read_csv("../competition_data.csv")

In [ ]:
#data cleaning
comp_data = comp_data.drop(
        columns=['accepts_mercadopago',
                # 'available_quantity',
                # 'avg_gmv_item_domain_30days',
                # 'avg_gmv_item_sel',
                # 'avg_gmv_seller_bday',
                # 'avg_qty_orders_item_domain_30days',
                # 'avg_qty_orders_item_sel_30days',
                # 'avg_si_item_sel_30day',
                'benefit',
                'boosted',
                'category_id',
                # 'conversion',
                'date',
                'deal_print_id',
                'domain_id',
                'etl_version',
                # 'free_shipping',
                # 'fulfillment',
                'full_name',
                # 'health',
                # 'is_pdp',
                'product_id',
                'item_id',
                # 'listing_type_id',
                # 'logistic_type',
                'main_picture',
                # 'offset',
                # 'original_price',
                'platform',
                # 'price',
                # 'print_position',
                'print_server_timestamp',
                # 'qty_items_dom',
                # 'qty_items_sel',
                'site_id',
                # 'sold_quantity',
                'tags',
                'title',
                # 'total_asp_item_domain_30days',
                # 'total_asp_item_sel_30days',
                # 'total_gmv_domain_bday',
                # 'total_gmv_item_30days',
                # 'total_items_domain',
                # 'total_items_seller',
                # 'total_orders_domain_30days',
                # 'total_orders_item_30days',
                # 'total_orders_sel_30days',
                # 'total_si_domain_30days',
                # 'total_si_item_30days',
                # 'total_si_sel_30days',
                # 'total_visits_domain',
                # 'total_visits_item',
                # 'total_visits_seller',
                'uid',
                'user_id',
                'warranty',
                # 'ROW_ID'
        ]
    )


In [ ]:
# full_data = comp_data[comp_data["ROW_ID"].isna()]
# eval_data = comp_data[comp_data["ROW_ID"].notna()]

# # Get number column names for full_data
# #print(full_data.columns)
# #num_cols = full_data.select_dtypes(include='number').columns
# #print(num_cols)
# # difference between full_data_cols and num_cols
# # print(set(full_data.columns) - set(num_cols))

# del comp_data
# gc.collect()

# train_data, test_data = train_test_split(full_data, test_size=0.20, train_size=0.80, random_state=42)

# y_train = train_data["conversion"]
# X_train = train_data.drop(columns=["conversion", "ROW_ID"])
# X_train = X_train.select_dtypes(include='number')

# y_test = test_data["conversion"]
# X_test = test_data.drop(columns=["conversion", "ROW_ID"])
# X_test = X_test.select_dtypes(include='number')

# del train_data
# del test_data

# gc.collect()

In [ ]:

train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number') ## retoque de datos para que haya mas
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=3456)
y_test = eval_data["conversion"]


#OHE comp_data columns
# comp_data = pd.get_dummies(comp_data, columns = [])

#OHE otra forma 
# Sobre training
X_train_obj = X_train.select_dtypes(include=["object"])
X_train_num = X_train.select_dtypes(include=["number"])

scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=True, handle_unknown="ignore")
X_train_ohe_obj = encoder.fit_transform(X_train_obj)
X_train_ohe_v2 = hstack([X_train_ohe_obj, scaler.fit_transform(X_train_num)])

# Sobre validation
X_val_obj = X_val.select_dtypes(include=["object"])
X_val_num = X_val.select_dtypes(include=["number"])

X_val_ohe_obj = encoder.transform(X_val_obj)
X_val_ohe_v2 = hstack([X_val_ohe_obj, scaler.transform(X_val_num)])



In [ ]:

#modelo de arbol
# Entrenamiento y evaluación del modelo Random Forest
rf = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=6789, verbose=1, oob_score=True)
rf.fit(pd.concat([X_train_ohe_v2, X_val_ohe_v2], axis=0),
       pd.concat([y_train, y_val], axis=0))
preds_test_rf = rf.predict_proba(X_test)[:, rf.classes_ == True]
print("ROC test score - Random Forest:", roc_auc_score(y_test, preds_test_rf)) # 0.9511696343957305

# Entrenamiento y evaluación del modelo XGBoost
params = {'colsample_bytree': 0.75,
               'gamma': 0.5,
               'learning_rate': 0.075,
               'max_depth': 8,
               'min_child_weight': 1,
               'n_estimators': 1200,
               'reg_lambda': 0.5,
               'subsample': 0.75,
               }

clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic',
                            seed = 1234,
                            eval_metric = 'auc',
                            **params)

clf_xgb.fit(X_train_ohe_v2, y_train, verbose = 100, eval_set = [(X_train_ohe_v2, y_train), (X_val_ohe_v2, y_val)])

preds_test_xgb = clf_xgb.predict_proba(X_test)[:, clf_xgb.classes_ == True]
print("AUC test score - XGBoost:", roc_auc_score(y_test, preds_test_xgb)) # 0.9719437378422864

cls = make_pipeline(SimpleImputer(), DecisionTreeClassifier(max_depth=8, random_state=2345))
cls.fit(X_train_ohe_v2, y_train)

# Predict on the evaluation set
# y = eval_data["conversion"]
eval_data = eval_data.drop(columns=["conversion"])
eval_data = eval_data.select_dtypes(include='number')
y_preds = cls.predict_proba(eval_data.drop(columns=["ROW_ID"]))[:, cls.classes_ == 1].squeeze()

# kf = KFold(n_splits=5, random_state=2345, shuffle=True)
# roc_aucs = []

# dtc = make_pipeline(
#     SimpleImputer(),
#     RandomizedSearchCV(
#         DecisionTreeClassifier(random_state=2345),
#         param_distributions={
#             "max_depth": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
#             "min_samples_split": [2, 4, 8, 16, 32, 64, 128, 256, 512],
#             "min_samples_leaf": [2, 4, 8, 16, 32, 64, 128, 256, 512],
#             "max_features": [2, 4, 8, 16, 32, 64, 128, 256, 512, None],
#         },
#         n_iter=100,
#         cv=3,
#         random_state=2345,
#         n_jobs=-1,
#         verbose=0,
#     )
# )

# for train_index, valid_index in kf.split(X_train):
#     X_train_kf, X_valid_kf = X_train.iloc[train_index], X_train.iloc[valid_index]
#     y_train_kf, y_valid_kf = y_train.iloc[train_index], y_train.iloc[valid_index]

#     dtc.fit(X_train_kf, y_train_kf)
#     # Metric: roc_auc_score
#     roc_aucs.append(roc_auc_score(y_valid_kf, dtc.predict_proba(X_valid_kf)[:, dtc.classes_ == 1]))
#     print(f"ROC-AUC split {len(roc_aucs)}: ", roc_aucs[-1])

# print("ROC-AUC mean: ", sum(roc_aucs) / len(roc_aucs))    

# dtc.fit(X_train, y_train)

# print("ROC-AUC: ", roc_auc_score(y_test, dtc.predict_proba(X_test)[:, dtc.classes_ == 1]))


In [ ]:
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": eval_data["ROW_ID"], "conversion": y_preds})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("basic_model_oli.csv", sep=",", index=False)

In [ ]:
roc_auc_score(y_val,y_preds)


## ----------------------------------------------------------------
# INTENTO 3

In [19]:
comp_data = pd.read_csv("./competition_data.csv")

In [20]:
train_data = comp_data[comp_data["ROW_ID"].isna()]
eval_data = comp_data[comp_data["ROW_ID"].notna()]
del comp_data
gc.collect()

y_train = train_data["conversion"]
X_train = train_data.drop(columns=["conversion", "ROW_ID"])
X_train = X_train.select_dtypes(include='number') ## retoque de datos para que haya mas
del train_data
gc.collect()

#hold out set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=3456)
y_test = eval_data["conversion"]

In [21]:
# Assuming 'X_train' is your DataFrame
atributos_ohe = []

for i, column_name in enumerate(X_train.columns):
    if isinstance(X_train[column_name][0], (float, int)) and column_name in menores_a_50:
        # atributos_ohe.append([ column_name, type(X_train[column_name][0])])
        atributos_ohe.append(column_name)
print(atributos_ohe)

# Create a DataFrame from the data
# table_df = pd.DataFrame(atributos_ohe, columns=['Column Name', 'Data Type'])

# Display the DataFrame as a table
# print(table_df)

['benefit', 'health']


In [22]:
X_train_ohe = pd.get_dummies(X_train,
                        columns = atributos_ohe,
                        sparse = True,    # Devolver una matriz rala.
                        dummy_na = False, # No agregar columna para NaNs.
                        dtype = int       # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )
# X_train_ohe

X_val_ohe = pd.get_dummies(X_val,                        
                        columns = atributos_ohe,
                        sparse = True,    # Devolver una matriz rala.
                        dummy_na = False, # No agregar columna para NaNs.
                        dtype = int       # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )

for columns_train, columns_val in zip(X_train_ohe.columns, X_val_ohe.columns):
    if columns_train != columns_val:
        print(False, columns_train, columns_val)


False health_0.36 health_0.4
False health_0.4 health_0.42
False health_0.42 health_0.44
False health_0.44 health_0.45
False health_0.45 health_0.5
False health_0.5 health_0.53


In [23]:
X_train_ohe.head()

,available_quantity,avg_gmv_item_domain_30days,avg_gmv_item_sel,avg_gmv_seller_bday,avg_qty_orders_item_domain_30days,avg_qty_orders_item_sel_30days,avg_si_item_sel_30day,product_id,offset,original_price,...,health_0.81,health_0.83,health_0.84,health_0.85,health_0.87,health_0.88,health_0.9,health_0.91,health_0.92,health_1.0
13223,20,203.751822,273.334509,1576.229000,3.637863,4.670520,5.063584,7571378.0,0,3199,...,0,0,0,0,0,1,0,0,0,0
121161,90398,573.574281,726.854545,266.513333,9.578684,32.454545,85.454545,NaN,384,650,...,0,0,0,0,0,0,0,0,0,1
9020,18,232.644091,180.850464,2206.375667,6.140316,8.374317,15.191257,NaN,144,1999,...,0,0,0,0,0,0,0,0,0,0
166398,234,59.801321,169.766875,181.084667,2.622240,13.500000,22.875000,NaN,192,840,...,0,0,0,0,0,0,0,0,0,0
95268,7,60.570196,195.463231,1694.014667,2.737064,4.188462,5.630769,NaN,192,680,...,0,0,0,0,0,0,0,0,0,0


In [24]:
testing = []
for i, column_name in enumerate(X_train_ohe.columns):
    if isinstance(X_train_ohe[column_name][0], (float, int)):
        # testing.append([ column_name, type(comp_data_ohe[column_name][0])])
        testing.append(column_name)
print(testing)

# # Create a DataFrame from the data
# table_df = pd.DataFrame(testing, columns=['Column Name', 'Data Type'])

# # Display the DataFrame as a table
# print(table_df)

['avg_gmv_item_domain_30days', 'avg_gmv_item_sel', 'avg_gmv_seller_bday', 'avg_qty_orders_item_domain_30days', 'avg_qty_orders_item_sel_30days', 'avg_si_item_sel_30day', 'product_id', 'qty_items_dom', 'qty_items_sel', 'total_asp_item_domain_30days', 'total_asp_item_sel_30days', 'total_gmv_domain_bday', 'total_gmv_item_30days', 'total_orders_domain_30days', 'total_orders_item_30days', 'total_orders_sel_30days', 'total_si_domain_30days', 'total_si_item_30days', 'total_si_sel_30days', 'user_id', 'benefit_0.0', 'health_0.36', 'health_0.4', 'health_0.42', 'health_0.44', 'health_0.45', 'health_0.5', 'health_0.54', 'health_0.55', 'health_0.57', 'health_0.58', 'health_0.6', 'health_0.61', 'health_0.62', 'health_0.63', 'health_0.66', 'health_0.69', 'health_0.7', 'health_0.71', 'health_0.72', 'health_0.75', 'health_0.76', 'health_0.77', 'health_0.8', 'health_0.81', 'health_0.83', 'health_0.84', 'health_0.85', 'health_0.87', 'health_0.88', 'health_0.9', 'health_0.91', 'health_0.92', 'health_1.0']

In [25]:
columns_to_erase = []
for columns in X_train_ohe:
    if columns not in testing:
        columns_to_erase.append(columns)

X_train_i3 = X_train_ohe.drop(columns_to_erase, axis=1)
X_val_i3 = X_val_ohe.drop(columns_to_erase, axis=1)

In [28]:
from scipy.stats import uniform
# Verifica si la columna 'health_0.53' está en X_train_i3
if 'health_0.53' not in X_train_i3.columns:
    # Si no está presente, agrégala y llénala con ceros
    X_train_i3['health_0.53'] = 0


params = {'max_depth': list(range(1, 40)),
          'learning_rate': uniform(scale=0.2),
          'gamma': uniform(scale=2),
          'reg_lambda': uniform(scale=5),
          'subsample': uniform(0.5, 0.5),
          'min_child_weight': uniform(scale=5),
          'colsample_bytree': uniform(0.75, 0.25),
          'n_estimators': list(range(1, 1000))
         }

random_state = 42
np.random.seed(random_state)

clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=random_state,
                            eval_metric='auc')

clf_xgb.fit(X_train_i3, y_train, verbose=True, eval_set=[(X_val_i3, y_val)])

XGBoostError: [00:58:20] /Users/runner/work/xgboost/xgboost/src/data/iterative_dmatrix.cc:93: Check failed: ref->Info().num_col_ == n_features (55 vs. 54) : Invalid ref DMatrix, different number of features.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000136ff6ab5 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000013710b752 xgboost::data::GetCutsFromRef(xgboost::Context const*, std::__1::shared_ptr<xgboost::DMatrix>, unsigned int, xgboost::BatchParam, xgboost::common::HistogramCuts*) + 418
  [bt] (2) 3   libxgboost.dylib                    0x0000000137108995 xgboost::data::IterativeDMatrix::InitFromCPU(xgboost::Context const*, xgboost::BatchParam const&, void*, float, std::__1::shared_ptr<xgboost::DMatrix>) + 5509
  [bt] (3) 4   libxgboost.dylib                    0x0000000137106d0c xgboost::data::IterativeDMatrix::IterativeDMatrix(void*, void*, std::__1::shared_ptr<xgboost::DMatrix>, void (*)(void*), in

# trash

In [27]:
start = time.time()
best_score = 0
best_estimator = None
iterations = 100
for g in ParameterSampler(params, n_iter = iterations, random_state = random_state):
    clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic', seed = random_state, eval_metric = 'auc', **g)
    clf_xgb.fit(X_train, Y_train, eval_set = [(X_val, Y_val)], verbose = False)

    y_pred = clf_xgb.predict_proba(X_val)[:, 1] # Obtenemos la probabilidad de una de las clases (cualquiera).
    auc_roc = sklearn.metrics.roc_auc_score(Y_val, y_pred)
    # Guardamos si es mejor.
    if auc_roc > best_score:
        print(f'Mejor valor de ROC-AUC encontrado: {auc_roc}')
        best_score = auc_roc
        best_grid = g
        best_estimator = clf_xgb

end = time.time()
print('ROC-AUC: %0.5f' % best_score)
print('Grilla:', best_grid)
print(f'Tiempo transcurrido: {str(end - start)} segundos')
print(f'Tiempo de entrenamiento por iteración: {str(round((end - start) / iterations, 2))} segundos')

NameError: name 'time' is not defined

In [ ]:
y_pred = best_estimator.predict_proba(X_test)[:, 1]
auc_roc = sklearn.metrics.roc_auc_score(Y_test, y_pred)
print('AUC-ROC test: %0.5f' % auc_roc)

## ----------------------------------------------------------------
# CODIGO AYUDA 
### KFOLD CLASE 3

In [ ]:
# k-fold cross validation
kcv = KFold(n_splits=10, shuffle=True)
rmse_kcv1 = []
for train_ix, test_ix in tqdm(kcv.split(X_train), total=10):
    X_train_red, X_val = X_train.iloc[train_ix,:], X_train.iloc[test_ix,:]
    y_train_red, y_val = y_train[train_ix], y_train[test_ix]
    tree = DecisionTreeRegressor(max_depth=10, random_state=TREE_SEED)
    tree.fit(X_train_red, y_train_red)
    preds_val = tree.predict(X_val)
    rmse_kcv1.append(math.sqrt(mean_squared_error(y_val, preds_val)))

print(f"RMSE estimado mediante k-fold CV (Versión 1): {mean(rmse_kcv1):.2f}")

# Validación cruzada k-fold utilizando cross_val_score
kcv = KFold(n_splits=10, shuffle=True)
tree = DecisionTreeRegressor(max_depth=10, random_state=TREE_SEED)
rmse_kcv2 = cross_val_score(tree, X_train, y_train, scoring='neg_root_mean_squared_error', cv=kcv, n_jobs=-1)

print(f"RMSE estimado mediante k-fold CV (Versión 2): {-1 * mean(rmse_kcv2):.2f}")


In [ ]:
#data cleaning
comp_data = comp_data.drop(
        columns=['accepts_mercadopago',
                # 'available_quantity',
                # 'avg_gmv_item_domain_30days',
                # 'avg_gmv_item_sel',
                # 'avg_gmv_seller_bday',
                # 'avg_qty_orders_item_domain_30days',
                # 'avg_qty_orders_item_sel_30days',
                # 'avg_si_item_sel_30day',
                'benefit',
                'boosted',
                'category_id',
                # 'conversion',
                'date',
                'deal_print_id',
                'domain_id',
                'etl_version',
                # 'free_shipping',
                # 'fulfillment',
                'full_name',
                # 'health',
                # 'is_pdp',
                'product_id',
                'item_id',
                # 'listing_type_id',
                # 'logistic_type',
                'main_picture',
                # 'offset',
                # 'original_price',
                'platform',
                # 'price',
                # 'print_position',
                'print_server_timestamp',
                # 'qty_items_dom',
                # 'qty_items_sel',
                'site_id',
                # 'sold_quantity',
                'tags',
                'title',
                # 'total_asp_item_domain_30days',
                # 'total_asp_item_sel_30days',
                # 'total_gmv_domain_bday',
                # 'total_gmv_item_30days',
                # 'total_items_domain',
                # 'total_items_seller',
                # 'total_orders_domain_30days',
                # 'total_orders_item_30days',
                # 'total_orders_sel_30days',
                # 'total_si_domain_30days',
                # 'total_si_item_30days',
                # 'total_si_sel_30days',
                # 'total_visits_domain',
                # 'total_visits_item',
                # 'total_visits_seller',
                'uid',
                'user_id',
                'warranty',
                # 'ROW_ID'
        ]
    )


In [ ]:
#hago XGboost


### Model selection
no se si usar


In [ ]:
# Training, holdout y testing sets
size_val = math.ceil(0.1 * X_train.shape[0])
size_test = math.ceil(0.1 * X_train.shape[0])

X_train_red, X_test, y_train_red, y_test = train_test_split(X_train, y_train, test_size=size_test)
X_train_red, X_val, y_train_red, y_val = train_test_split(X_train_red, y_train_red, test_size=size_val)

# Prueba con distintos niveles de profundidad (model selection)
exp_results = []
for md in tqdm(range(1, 51)):
    tree = DecisionTreeRegressor(max_depth=md, random_state=TREE_SEED)
    tree.fit(X_train_red, y_train_red)
    preds_val = tree.predict(X_val)
    exp_results.append({"max_depth": md,
                        "rmse_val": math.sqrt(mean_squared_error(y_val, preds_val))})

exp_results = pd.DataFrame(exp_results)
plot_exp(exp_results)

# Se entrena el árbol con la mejor profundidad encontrada sobre train set + validation set
best_md = exp_results[exp_results["rmse_val"].min() == exp_results["rmse_val"]]
best_md = best_md.sort_values("max_depth").iloc[0,:]
print(f"Performance del mejor modelos: {best_md['rmse_val']:.2f}")

tree = DecisionTreeRegressor(max_depth=int(best_md["max_depth"]), random_state=TREE_SEED)
tree.fit(pd.concat([X_train_red, X_val], axis=0),
         pd.concat([y_train_red, y_val], axis=0))
preds_test = tree.predict(X_test)

print(f"RMSE estimado en test: {math.sqrt(mean_squared_error(y_test, preds_test)):.2f}")

# Evaluación final en el conjunto de evaluación (model assestment)
tree = DecisionTreeRegressor(max_depth=int(best_md["max_depth"]), random_state=TREE_SEED)
tree.fit(X_train, y_train)
preds_eval = tree.predict(X_eval)
_, _, _, y_eval = load_data()

print(f"Performance de evaluación: {math.sqrt(mean_squared_error(y_eval, preds_eval)):.2f}")

### OHE

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, precision_recall_fscore_support, precision_recall_curve, average_precision_score
import numpy as np
from tqdm import tqdm
# One-hot-encoding
# Opción 1: utilizar get_dummies de pandas
X_train_ohe_v1 = pd.get_dummies(X_train).head()  # Y ahora cómo lo aplico en validación? (debería unir antes los datos)

# Opción 2: utilizar onehotencoder de sklearn (más conveniente para producción)

# Sobre training
X_train_obj = X_train.select_dtypes(include=["object"])
X_train_num = X_train.select_dtypes(include=["int", "float"])

scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=True)
X_train_ohe_obj = encoder.fit_transform(X_train_obj)
X_train_ohe_v2 = hstack([X_train_ohe_obj, scaler.fit_transform(X_train_num)])

# Sobre validation
X_val_obj = X_val.select_dtypes(include=["object"])
X_val_num = X_val.select_dtypes(include=["int", "float"])

X_val_ohe_obj = encoder.transform(X_val_obj)
X_val_ohe_v2 = hstack([X_val_ohe_obj, scaler.transform(X_val_num)])

# Entrenamos un modelo de regresión logística sobre los datos con OHE sin escalado
lr_exp = []
for C in tqdm([0.001, 0.0025, 0.01, 0.05, 0.075, 0.1, 0.25]):
    lr = LogisticRegression(C=C, max_iter=2000)
    lr.fit(X_train_ohe_v2, y_train)
    preds = lr.predict_proba(X_val_ohe_v2)
    preds = preds[:, lr.classes_ == "yes"]
    auc = roc_auc_score(y_val == "yes", preds)
    lr_exp.append({"C": C, "auc_val": auc})

lr_exp = pd.DataFrame(lr_exp)
print(lr_exp)